In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_tr=pd.read_csv("../input/forest-cover-type-prediction/train.csv")
df_te=pd.read_csv("../input/forest-cover-type-prediction/test.csv")

In [ ]:
df_tr.head()

In [ ]:
df_tr.isnull().sum()

In [ ]:
df_tr.shape

In [ ]:
sns.heatmap(df_tr.corr())

In [ ]:
df_tr["Wilderness_Area1"].value_counts()

In [ ]:
df_tr["Wilderness_Area1"].unique()

In [ ]:
for col in df_tr.columns:
    if col=="Id":
        print("")
    elif col=="Elevation" or col=="Aspect" or col=="Slope" or col=="Horizontal_Distance_To_Hydrology" or col=="Vertical_Distance_To_Hydrology" or col=="Horizontal_Distance_To_Roadways" or col=="Hillshade_9am" or col=="Hillshade_Noon" or col=="Hillshade_3pm" or col=="Horizontal_Distance_To_Fire_Points" :
        plt.hist(df_tr[col].value_counts(),Label=True)
        plt.title('Histogram for {}'.format(col))
        plt.show(); 
    else:
        plt.pie(df_tr[col].value_counts(),labels=df_tr[col].unique())
        plt.title('Piechart for {}'.format(col))
        plt.show();

In [ ]:
sns.pairplot(data=df_tr[["Elevation","Aspect","Slope","Horizontal_Distance_To_Hydrology","Vertical_Distance_To_Hydrology","Horizontal_Distance_To_Roadways","Hillshade_9am","Hillshade_Noon","Hillshade_3pm","Horizontal_Distance_To_Fire_Points","Cover_Type"]],hue="Cover_Type")

In [ ]:
for col in df_tr.columns:
    if col=="Id" or col=="Elevation" or col=="Aspect" or col=="Slope" or col=="Horizontal_Distance_To_Hydrology" or col=="Vertical_Distance_To_Hydrology" or col=="Horizontal_Distance_To_Roadways" or col=="Hillshade_9am" or col=="Hillshade_Noon" or col=="Hillshade_3pm" or col=="Horizontal_Distance_To_Fire_Points" or col=="Cover_Type":
        sns.distplot(df_tr[col])
        plt.title('Distplot for {}'.format(col))
        plt.show();
    else:
        sns.boxplot(x=col, y='Cover_Type', data=df_tr)
        plt.title('Boxplot for {}'.format(col))
        plt.show();

In [ ]:
x=df_tr.iloc[:,1:55]
y=df_tr.Cover_Type

In [ ]:
x_te=df_te.iloc[:,1:]

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(x,y,test_size = .2,random_state =42)
x_train.shape

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [ ]:
xrf_pipe = Pipeline(
    steps = [
        ('classifier', ExtraTreesClassifier(n_estimators=500,random_state=0, criterion = 'entropy'))
    ]
)


xrf_param_grid = {
    'classifier__min_samples_leaf': [1,4,7],
    'classifier__max_depth': [34,38,32],
}

np.random.seed(1)
xrf_grid_search = GridSearchCV(xrf_pipe, xrf_param_grid, cv=5, refit='True', n_jobs=-1)
xrf_grid_search.fit(x_train, y_train)
print(xrf_grid_search.best_score_)
print(xrf_grid_search.best_params_)

In [ ]:
xrf_model = xrf_grid_search.best_estimator_
cv_score = cross_val_score(xrf_model, x_val, y_val, cv = 5)
cv_score

In [ ]:
final_model = xrf_grid_search.best_estimator_.steps[0][1]
final_model.fit(x_train, y_train)

In [ ]:
y_pred = final_model.predict(x_te)
y_pred
#74%accuracy